# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

# Apple 🍎

Ao iniciar nosso projeto de Ciência dos Dados, tinhamos como objetivo aplicar conceitos vistos em sala, como o Classificador Naive Bayes, Laplace Smothing, entre outros. Para isso, foi proposto que, com a ajuda do ``Python`` e algumas de suas bibliotecas, fizessemos nosso próprio classificador de sentimentos de alguma empresa, com base em tweets obtidos através de uma API disponibilizada. Com isso escolhemos fazer um classificador de relevância da empresa ``Apple`` ``(relevante / irrelevante)`` na ``linguagem inglesa``, por gerar muita ``ambiguidade entre a fruta e a empresa`` em si, caracterizando um problema provavelmente muito recorrente para recebimento de feedbacks.

### Import de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

### Funções do Classificador

In [36]:

def limpeza_de_dados(treinamento, coluna):
    dados_limpos = []
    for e in treinamento[coluna]:
        limpeza = e.replace('\n', '')
        limpeza = str(limpeza)
        limpeza = limpeza.replace('t.co', '')
        limpeza = limpeza.replace('https://', '')
        limpeza = limpeza.replace('#', '')
        limpeza = limpeza.replace(':', '')
        limpeza = limpeza.replace(',', '')
        limpeza = limpeza.replace('.', '')
        limpeza = limpeza.replace('/', '')
        limpeza = limpeza.replace('//', '')
        limpeza = limpeza.replace('...', '')
        limpeza = limpeza.replace(';', '')
        limpeza = limpeza.replace('\'', '')
        limpeza = limpeza.replace('rt', '')
        limpeza = limpeza.replace('@', '')
        limpeza = limpeza.lower()
        limpeza = limpeza.split(' ')
        dados_limpos.append(limpeza)

    return dados_limpos

def contagem_palavras(dados_limpos):
    contagem = {}
    for e in dados_limpos:
        for all in e:
            if all not in contagem:
                contagem[all] = 1
            else:
                contagem[all] += 1
    del contagem['']
    
    return contagem

def frequencia_das_palavras(contagem):
    probabilidade_palavra = {}
    soma = sum(contagem.values())
    for k in contagem:
        probabilidade_palavra[k] = contagem[k] / soma

    return probabilidade_palavra

def laplace_smoothing(probabilidade_palavra, prob_total):
    laplace_probabilidade_palavra = {}
    for k in probabilidade_palavra:
        laplace_probabilidade_palavra[k] = np.log( (probabilidade_palavra[k]+1) / prob_total )
        
    return laplace_probabilidade_palavra

def calculador_probabilidade(frases_tratadas, laplace_probabilidade_positivo, laplace_probabilidade_negativo, probabilidade_positivo_total, probabilidade_negativo_total):
    resultado = []
    for e in frases_tratadas: #loop frase
        conta_positivo = []
        conta_negativo = []
        for all in e: #loop palavras da frase
            for k in laplace_probabilidade_positivo: #loop nas frequencias positivas
                if all in k: #se a palavra estiver no dicionario de frequencias positivas
                    conta_positivo.append(laplace_probabilidade_positivo[k]) #adicionar a lista conta_positivo a probabilidade da palavra k
            for k in laplace_probabilidade_negativo:
                if all in k:
                    conta_negativo.append(laplace_probabilidade_negativo[k])
        prob_positivo = np.prod(conta_positivo) * probabilidade_positivo_total
        prob_negativo = np.prod(conta_negativo) * probabilidade_negativo_total
        if prob_positivo > prob_negativo:
            resultado.append(1)
        else:
            resultado.append(0)
    return resultado

## Treinamento do Classificador

### Leitura / Limpeza / Contagem dos Dados

In [45]:
#Leitura dos daods de treinamento
leitura_excel_treinamento = pd.read_excel('tweets_apple.xlsx', sheet_name='Treinamento')

# Divisao dos dados entre relevante e irrelevante (positivo/negativo)
treinamento_raw_positivo = leitura_excel_treinamento[leitura_excel_treinamento.Relevancia==1]
treinamento_raw_negativo = leitura_excel_treinamento[leitura_excel_treinamento.Relevancia==0]

# Limpeza dos dados (positivo/negativo)
limpeza_positivo = limpeza_de_dados(treinamento_raw_positivo, 'Treinamento')
limpeza_negativo = limpeza_de_dados(treinamento_raw_negativo, 'Treinamento')

# Contagem das palavras (positivo/negativo)
contagem_positivo = contagem_palavras(limpeza_positivo)
contagem_negativo = contagem_palavras(limpeza_negativo)

### Aplicação de Naive Bayes e Laplace Smoothing

In [38]:
# Probabilidades Totais (total / total positiva / total negativa)
prob_total = len(treinamento_raw_positivo) + len(treinamento_raw_negativo)
probabilidade_positivo_total = np.log(len(treinamento_raw_positivo) + 1 / (len(treinamento_raw_positivo) + len(treinamento_raw_negativo)))
probabilidade_negativo_total = np.log(len(treinamento_raw_negativo) + 1 / (len(treinamento_raw_positivo) + len(treinamento_raw_negativo)))

# Probabilidade de cada palavra dado (positivo / negativo)
frequencia_palavra_positivo = frequencia_das_palavras(contagem_positivo)
frequencia_palavra_negativo = frequencia_das_palavras(contagem_negativo)

# Aplicacao de Laplace smoothing nos dados (positivo/negativo)
laplace_probabilidade_positivo = laplace_smoothing(frequencia_palavra_positivo, prob_total)
laplace_probabilidade_negativo = laplace_smoothing(frequencia_palavra_negativo, prob_total)

## Teste do Classificador

### Limpeza / Calculo / Classificacao dos Dados

In [44]:
#Leitura dos dados de teste
leitura_excel_teste = pd.read_excel('tweets_apple.xlsx', sheet_name='Teste')

#Limpeza dos dados
teste_tratado = limpeza_de_dados(leitura_excel_teste, 'Teste')
teste_tratado


a=calculador_probabilidade(teste_tratado, laplace_probabilidade_positivo,laplace_probabilidade_negativo, probabilidade_positivo_total, probabilidade_negativo_total)
leitura_excel_teste['batata'] = a
leitura_excel_teste



112
87
1.2873563218390804


F:\anaconda\lib\site-packages\numpy\core\_methods.py:35: RuntimeWarning: overflow encountered in reduce
  return umr_prod(a, axis, dtype, out, keepdims)


### Comparação de Dados

In [46]:
batata = limpeza = leitura_excel_teste.batata.sum()
rel = limpeza = leitura_excel_teste.Relevancia.sum()
print(batata)
print(rel)
print(112/87)

112
87
1.2873563218390804


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
